# Homeworh 5 - part 2

## ChEBI SQLALchemy model

7. Use the [ER model](http://ftp.ebi.ac.uk/pub/databases/chebi/DataModel.png) and [SQL database model](http://ftp.ebi.ac.uk/pub/databases/chebi/generic_dumps/mysql_create_tables.sql) to create your SQLAlchemy

8. Design [ChEBI](https://www.ebi.ac.uk/chebi/) database as SQLALchemy model

9. Create same example queries with SQLAlchemy like in the exercises.

design sqlalchemy model based on .sql file

In [ ]:
#
# Table structure for table 'chemical_data'
#

DROP TABLE IF EXISTS `chemical_data`;
CREATE TABLE `chemical_data` (
  `id` INT NOT NULL,
  `compound_id` INT NOT NULL,
  `chemical_data` TEXT NOT NULL,
  `source` TEXT NOT NULL,
  `type` TEXT NOT NULL,
  PRIMARY KEY (`id`)
) ENGINE=InnoDB;


#
# Table structure for table 'comments'
#

DROP TABLE IF EXISTS `comments`;
CREATE TABLE `comments` (
  `id` INT NOT NULL,
  `compound_id` INT NOT NULL,
  `text` TEXT NOT NULL,
  `created_on` DATETIME NOT NULL,
  `datatype` VARCHAR(80),
  `datatype_id` INT NOT NULL,
  PRIMARY KEY (`id`)
) ENGINE=InnoDB;


#
# Table structure for table 'compounds'
#

DROP TABLE IF EXISTS `compounds`;
CREATE TABLE `compounds` (
  `id` INT NOT NULL,
  `name` TEXT,
  `source` VARCHAR(32) NOT NULL,
  `parent_id` INT,
  `chebi_accession` VARCHAR(30) NOT NULL,
  `status` VARCHAR(1) NOT NULL,
  `definition` TEXT,
  `star` INT NOT NULL,
  `modified_on` TEXT,
  `created_by` TEXT,
  PRIMARY KEY (`id`)
) ENGINE=InnoDB;


#
# Table structure for table 'database_accession'
#

DROP TABLE IF EXISTS `database_accession`;
CREATE TABLE `database_accession` (
  `id` INT NOT NULL,
  `compound_id` INT NOT NULL,
  `accession_number` VARCHAR(255) NOT NULL,
  `type` TEXT NOT NULL,
  `source` TEXT NOT NULL,
  PRIMARY KEY (`id`)
) ENGINE=InnoDB;


#
# Table structure for table 'names'
#

DROP TABLE IF EXISTS `names`;
CREATE TABLE `names` (
  `id` INT NOT NULL,
  `compound_id` INT NOT NULL,
  `name` TEXT NOT NULL,
  `type` TEXT NOT NULL,
  `source` TEXT NOT NULL,
  `adapted` TEXT NOT NULL,
  `language` TEXT NOT NULL,
  PRIMARY KEY (`id`)
) ENGINE=InnoDB;


#
# Table structure for table 'reference'
#

DROP TABLE IF EXISTS `reference`;
CREATE TABLE `reference` (
  `id` INT NOT NULL,
  `compound_id` INT NOT NULL,
  `reference_id` VARCHAR(60) NOT NULL,
  `reference_db_name` VARCHAR(60) NOT NULL,
  `location_in_ref` VARCHAR(90),
  `reference_name` VARCHAR(1024),
  PRIMARY KEY (`id`)
) ENGINE=InnoDB;


#
# Table structure for table 'relation'
#

DROP TABLE IF EXISTS `relation`;
CREATE TABLE `relation` (
  `id` INT NOT NULL,
  `type` TEXT NOT NULL,
  `init_id` INT NOT NULL,
  `final_id` INT NOT NULL,
  `status` VARCHAR(1) NOT NULL, 
  PRIMARY KEY (`id`)
) ENGINE=InnoDB;


#
# Table structure for table 'structures'
#

DROP TABLE IF EXISTS `structures`;
CREATE TABLE `structures` (
  `id` INT NOT NULL,
  `compound_id` INT NOT NULL,
  `structure` TEXT NOT NULL,
  `type` TEXT NOT NULL,
  `dimension` TEXT NOT NULL,
  `default_structure` VARCHAR(1) NOT NULL,
  `autogen_structure` VARCHAR(1) NOT NULL,
  PRIMARY KEY (`id`)
) ENGINE=InnoDB;


#
# Table structure for table 'compound_origins'
#

DROP TABLE IF EXISTS `compound_origins`;
CREATE TABLE `compound_origins` (
  `id` INT NOT NULL,
  `compound_id` INT NOT NULL,
  `species_text` TEXT NOT NULL,
  `species_accession` TEXT,
  `component_text` TEXT,
  `component_accession` TEXT,
  `strain_text` TEXT,
  `source_type` TEXT NOT NULL,
  `source_accession` TEXT NOT NULL,
  `comments` TEXT,
  PRIMARY KEY (`id`)
) ENGINE=InnoDB;


ALTER TABLE `chemical_data` ADD INDEX (`compound_id`);
ALTER TABLE `chemical_data` ADD FOREIGN KEY `FK_CHEMICAL_DATA_TO_COMPOUND`(`compound_id`) REFERENCES `compounds`(`id`);
ALTER TABLE `comments` ADD INDEX (`compound_id`);
ALTER TABLE `comments` ADD FOREIGN KEY `FK_COMMENTS_TO_COMPOUND`(`compound_id`) REFERENCES `compounds`(`id`);
ALTER TABLE `compound_origins` ADD INDEX (`compound_id`);
ALTER TABLE `compound_origins` ADD FOREIGN KEY `FK_COMPOUND_ORIGINS_TO_COMPOUND`(`compound_id`) REFERENCES `compounds`(`id`);
ALTER TABLE `database_accession` ADD INDEX (`compound_id`);
ALTER TABLE `database_accession` ADD FOREIGN KEY `FK_DB_ACCESSION_TO_COMPOUND`(`compound_id`) REFERENCES `compounds`(`id`);
ALTER TABLE `names` ADD INDEX (`compound_id`);
ALTER TABLE `names` ADD FOREIGN KEY `FK_NAMES_COMPOUND`(`compound_id`) REFERENCES `compounds`(`id`);
ALTER TABLE `reference` ADD INDEX (`compound_id`);
ALTER TABLE `reference` ADD FOREIGN KEY `FK_REFERENCE_TO_COMPOUND`(`compound_id`) REFERENCES `compounds`(`id`);
ALTER TABLE `relation` ADD INDEX (`final_id`);
ALTER TABLE `relation` ADD FOREIGN KEY `FK_RELATION_TO_FINAL_COMPOUND_ID`(`final_id`) REFERENCES `compounds`(`id`);
ALTER TABLE `relation` ADD INDEX (`init_id`);
ALTER TABLE `relation` ADD FOREIGN KEY `FK_RELATION_TO_INIT_COMPOUND_ID`(`init_id`) REFERENCES `compounds`(`id`);
ALTER TABLE `structures` ADD FOREIGN KEY `FK_STRUCTURES_TO_COMPOUND`(`compound_id`) REFERENCES `compounds`(`id`);


In [7]:
from sqlalchemy.orm import relationship
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Integer, String, Table, Text, ForeignKey, DateTime


Base = declarative_base()


class Compounds(Base):
    
    __tablename__ = 'compounds'
    
    id = Column(Integer, nullable=False, primary_key=True)
    name = Column(Text)
    source = Column(String(32), nullable=False)
    parent_id = Column(Integer)
    chebi_accession = Column(String(30), nullable=False)
    status = Column(String(1), nullable=False)
    defination = Column(Text)
    star = Column(Integer, nullable=False)
    modified_on = Column(Text)
    created_by = Column(Text)
    
    chemical_data_rel = relationship('ChemicalData', back_populates='compound')
    comments_rel = relationship('Comments', back_populates='compound')
    compound_origins_rel = relationship('CompoundOrigins', back_populates='compound')
    database_accession_rel = relationship('DatabaseAccession', back_populates='compound')
    names_rel = relationship('Names', back_populates='compound')
    relation_rel = relationship('Relation', back_populates='compound')

def as_dict(self):
        """Convert object values to dictionary."""
        xrefs_grouped = defaultdict(list)
        for xref in self.xrefs:
            xrefs_grouped[xref.db].append(xref.identifier)
        xrefs_grouped = {k: sorted(v) for k, v in xrefs_grouped.items()}

        return {
            'name': self.name,
            'accession': self.accession,
            'recommended_name': self.recommended_name,
            'taxid': self.taxid,
            'function_description': self.function.description if self.function else self.function,
            'gene_names': [x.name for x in self.gene_names],
            'gene_symbol': self.gene_symbol.symbol if self.gene_symbol else self.gene_symbol,
            'keywords': [{'keyword': x.keyword_name, 'id': x.keywordid} for x in self.keywords],
            'xrefs': xrefs_grouped,
            'subcellular_locations': [x.name for x in self.subcellular_locations],
            'organism': self.organism.scientific_name
        }


class ChemicalData(Base):
    
    __tablename__ = 'chemical_data'
    
    id = Column(Integer, nullable=False, primary_key=True)
    compound_id = Column(Integer, ForeignKey('compounds.id'), nullable=False)
    chemical_data = Column(Text, nullable=False)
    source = Column(Text, nullable=False)
    type = Column(Text, nullable=False)
    
    compound = relationship('Compounds', back_populates='chemical_data_rel')
    

class Comments(Base):
    
    __tablename__ = 'comments'
    
    id = Column(Integer, nullable=False, primary_key=True)
    compound_id = Column(Integer, ForeignKey('compounds.id'), nullable=False)
    text = Column(Text, nullable=False)
    created_on = Column(DateTime, nullable=False)
    datatype = Column(String(80))
    datatype_id = Column(Integer, nullable=False)
    
    compound = relationship('Compounds', back_populates='comments_rel')

class CompoundOrigins(Base):
    
    __tablename__ = 'compound_origins'
    
    id = Column(Integer, nullable=False, primary_key=True)
    compound_id = Column(Integer, ForeignKey('compounds.id'), nullable=False)
    species_text = Column(Text, nullable=False)
    species_accession = Column(Text)
    component_text = Column(Text)
    component_accession = Column(Text)
    strain_text = Column(Text)
    source_type = Column(Text, nullable=False)
    source_accession = Column(Text, nullable=False)
    comments = Column(Text)
    
    compound = relationship('Compounds', back_populates='compound_origins_rel')
    

class DatabaseAccession(Base):
    
    __tablename__ = 'database_accession'
    
    id = Column(Integer, nullable=False, primary_key=True)
    compound_id = Column(Integer, ForeignKey('compounds.id'), nullable=False)
    accession_number = Column(String(255), nullable=False)
    type = Column(Text, nullable=False)
    source = Column(Text, nullable=False)
    
    compound = relationship('Compounds', back_populates='database_accession_rel')

class Names(Base):
    
    __tablename__ = 'names'
    
    id = Column(Integer, nullable=False, primary_key=True)
    compound_id = Column(Integer, ForeignKey('compounds.id'), nullable=False)
    name = Column(Text, nullable=False)
    type = Column(Text, nullable=False)
    source = Column(Text, nullable=False)
    adapted = Column(Text, nullable=False)
    laguage = Column(Text, nullable=False)
    
    compound = relationship('Compounds', back_populates='names_rel')

class Relation(Base):
    
    __tablename__ = 'relation'
    
    id = Column(Integer, nullable=False, primary_key=True)
    type = Column(Text, nullable=False)
    init_id = Column(Integer, ForeignKey('compounds.id'), nullable=False)
    final_id = Column(Integer, ForeignKey('compounds.id'), nullable=False)
    status = Column(String(1), nullable=False)
    
    compound = relationship('Compounds', back_populates='relation_rel')
    